# Ejecuta los códigos en TensorFlow 2.0 sin editar los de la versión 1.x (excepto el módulo contrib).

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

###### 1.1. Leer datos y preprocesar:

In [3]:
# Leer datos.
data_raw = load_iris()
X = data_raw['data']
y = data_raw['target'].reshape(-1,1)

In [ ]:
# X

In [4]:
# Escalamiento:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
# Convierte en un DataFrame y visualiza.
header = ['SepalLength','SepalWidth','PetalLength','PetalWidth']      # nombre de columnas
df = pd.DataFrame(X_scaled,columns=header)
df.head(3)

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,0.222222,0.625000,0.067797,0.041667
1,0.166667,0.416667,0.067797,0.041667
2,0.111111,0.500000,0.050847,0.041667


In [ ]:
df.describe().loc[['min','max']]

###### 1.2. Mostrar el resultado de PCA:

In [ ]:
# Reduce a 2 dimensiones.
pca = PCA(n_components = 2)
X_pca = pca.fit_transform(X_scaled)

In [ ]:
sum(pca.explained_variance_ratio_)

In [ ]:
pd.Series(data_raw['target']).value_counts()

In [10]:
# Convierte las etiquetas de números enteros en nombres de colores.
mycolor = []
for i in y:
    if i == 0:
        mycolor.append('red')
    elif i == 1:
        mycolor.append('green')
    else:
        mycolor.append('blue')

In [ ]:
# Visualiza
plt.scatter(X_pca[:,0],X_pca[:,1],marker="o",alpha=0.7, s=10, c=mycolor)
plt.show()

##### 2. Reducción dimensional con AutoEncoder:
###### 2.1. Define un modelo de AutoEncoder:

In [ ]:
# Definición de hiperparámetros
n_input = 4             # Las capas de entrada tienen tantos nodos como variables.
n_hidden = 2            # El número de nodos en la capa oculta = 2 <= mi objetivo.
n_output = n_input      # La salida debe tener la misma cantidad de nodos que la entrada.
learn_rate = 0.0001
n_epochs = 10001

In [ ]:
# Definición de variables
initializer = tf.variance_scaling_initializer(seed = 12345)  ## Inicializar aleatoriamente los pesos
W1 = tf.Variable(initializer([n_input, n_hidden]), dtype=tf.float32)
W2 = tf.Variable(initializer([n_hidden, n_output]), dtype=tf.float32)
b1 = tf.Variable(tf.zeros(n_hidden))
b2 = tf.Variable(tf.zeros(n_output))

In [ ]:
W2

In [ ]:
# Definición de marcador de posición
X_ph = tf.placeholder(tf.float32, shape=[None, n_input])

In [ ]:
X_ph

In [ ]:
# Definición del modelo AutoEncoder
hidden_layer = tf.matmul(X_ph, W1) + b1                  # No activación.
y_model = tf.matmul(hidden_layer, W2) + b2
loss = tf.reduce_mean(tf.square(X_ph - y_model))         # 'X' ocupa el lugar de 'y'.
optimizer = tf.train.AdamOptimizer(learning_rate = learn_rate)
train = optimizer.minimize(loss)
init = tf.global_variables_initializer()

###### 2.2. Entrenamiento de AutoEncoder:

In [ ]:
%%time
with tf.Session() as sess:
        sess.run(init)
        for i in range(n_epochs):
            my_feed = {X_ph:X_scaled}
            sess.run(train, feed_dict = my_feed)
            if i % 1000 == 0:
                mse = sess.run(loss, feed_dict = my_feed)
                print("Step : {}    ,   MSE  : {}".format(i, mse))
        X_auto = sess.run(hidden_layer, feed_dict = my_feed)       # ¡¡¡Obtén la representación dimensional reducida de la capa oculta!!!


###### 2.3. Comparemos los resultados de PCA frente a AutoEncoder:

In [ ]:
fig=plt.figure(figsize=(10,5), dpi=80)       # tamaño de figura = (Ancho, Alto). Establecer DPI
axes1 = fig.add_axes([0,0,0.4,1])            # Izquierda, Abajo, Ancho, Alto
axes2 = fig.add_axes([0.5,0,0.4,1])          # Izquierda, Abajo, Ancho, Alto
axes1.scatter(X_pca[:,0],X_pca[:,1],marker="o",alpha=0.7, s=10, c=mycolor)
axes2.scatter(X_auto[:,0],X_auto[:,1],marker="o",alpha=0.7, s=10, c=mycolor)
axes1.set_xlabel('X')
axes1.set_ylabel('Y')
axes1.set_title('PCA')
axes2.set_xlabel('X')
axes2.set_ylabel('Y')
axes2.set_title('AutoEncoder')
plt.show()

Ahora probamos con la libreria de Keras, construyendo una red neuronal con Keras. (Reiniciar el kernel)

In [14]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

batch_size = 20
learn_rate = 0.0001
n_epochs = 10001

my_model = tf.keras.Sequential()
my_model.add(tf.keras.layers.Dense(2, input_shape=(4,)))
my_model.add(tf.keras.layers.Dense(4))


my_optimizer=Adam(learning_rate=learn_rate)
my_model.compile(loss = "MeanSquaredError", optimizer = my_optimizer)

my_model.summary()

c:\Users\mcruz\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 2)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22 (88.00 B)

 Trainable params: 22 (88.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
%%time
my_summary = my_model.fit(X_scaled, X_scaled, epochs=1000, batch_size = batch_size, verbose = 1)
# my_model.summary()

Epoch 1/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6580 
Epoch 2/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6851 
Epoch 3/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7167 
Epoch 4/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 857us/step - loss: 0.6106
Epoch 5/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6885 
Epoch 6/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6219 
Epoch 7/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6260 
Epoch 8/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - loss: 0.6592
Epoch 9/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6205 
Epoch 10/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6557 
Epoch 11/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6087
Epoch 12/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6370 
Epoch 13/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5976 
Epoch 14/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6273  
Epoch 15/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6183 
Ep

In [15]:
#my_model.predict(X_scaled)


from keras.models import Model


intermediate_layer_model = Model(inputs=my_model.input, outputs=my_model.get_layer("dense_3").output)
intermediate_output = intermediate_layer_model.predict(X_scaled)

print(intermediate_output)

ValueError: The layer sequential_2 has never been called and thus has no defined input.

In [ ]:
intermediate_output.shape

In [ ]:
fig=plt.figure(figsize=(10,5), dpi=80)       # tamaño de figura = (Ancho, Alto). Establecer DPI
axes1 = fig.add_axes([0,0,0.4,1])            # Izquierda, Abajo, Ancho, Alto

axes1.scatter(intermediate_output[:,0],intermediate_output[:,1],marker="o",alpha=0.7, s=10, c=mycolor)

axes1.set_xlabel('X')
axes1.set_ylabel('Y')
axes1.set_title('Keras AutoEncoder')

plt.show()